In [1]:
import json
import pandas as pd
import re


In [2]:
with open('TLP.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [3]:
df = pd.DataFrame(data)

In [4]:
# Hàm loại bỏ dấu chấm trong cột battery
df['battery'] = df['battery'].str.replace('.', '', regex=False)
df['battery'] = df['battery'].str.replace(',', '', regex=False)


In [5]:
# Hàm xử lý dữ liệu pin
def clean_battery(value):
    # Tìm dung lượng pin (xxxx mAh) trong chuỗi
    match = re.search(r'\d{3,5}\s*mAh', str(value))  # Tìm các số có từ 3-5 chữ số theo sau là 'mAh'
    if match:
        return match.group(0)  # Nếu tìm thấy, trả về phần dung lượng pin
    else:
        return 'N/A'  # Nếu không, thay thế bằng 'N/A'

# Áp dụng hàm vào cột battery
df['battery'] = df['battery'].apply(clean_battery)


In [6]:
# Hàm tách giá trị và đơn vị, xử lý ngoại lệ
def split_value_unit(column):
    # Regular expression để tách số khỏi đơn vị
    value = column.apply(lambda x: re.findall(r'[\d,.]+', str(x))[0] if pd.notnull(x) and re.findall(r'[\d,.]+', str(x)) else "N/A")  # Tách phần số
    unit = column.apply(lambda x: re.sub(r'[\d,.]+', '', str(x)).strip() if pd.notnull(x) and re.findall(r'[\d,.]+', str(x)) else "N/A")  # Tách phần đơn vị
    return value, unit


# Áp dụng hàm cho các cột 'battery' và 'display_size'
df['battery_value'], df['battery_unit'] = split_value_unit(df['battery'])
df['display_size_value'], df['display_size_unit'] = split_value_unit(df['display_size'])
df['mobile_tan_so_quet_value'], df['mobile_tan_so_quet_unit'] = split_value_unit(df['mobile_tan_so_quet'])


In [8]:

# Hàm để chuyển đổi định dạng rating từ "x/y" thành "x.0/y"
def format_rating(rating):
    if pd.isna(rating):
        return rating  # Trả về nguyên bản nếu là NaN
    try:
        # Tách giá trị, làm tròn số đến 1 chữ số thập phân
        score, out_of = rating.split('/')
        return f"{float(score):.1f}/{out_of}"
    except:
        return rating  # Trả về nguyên bản nếu có lỗi

# Áp dụng hàm vào cột `average_rating`
df['average_rating'] = df['average_rating'].apply(format_rating)

In [9]:
# Hàm thay thế giá trị 0 trong special_price bằng giá trị tương ứng trong price
df['special_price'] = df.apply(
    lambda row: row['price'] if row['special_price'] == 0 else row['special_price'], axis=1)


In [10]:
#xóa cột battery và display_size
df.drop(['battery', 'display_size', 'mobile_tan_so_quet'], axis=1, inplace=True)


In [11]:
#Sắp xếp lại thứ tự cột
df = df[['product_id',	'name',	'price',	'special_price', 'battery_value',	'battery_unit',	'display_size_value',	'display_size_unit',	'mobile_tan_so_quet_value', 'mobile_tan_so_quet_unit',	'manufacturer',	'chipset',	'mobile_tinh_nang_dac_biet',	'mobile_cong_sac',	'warranty_information',	'average_rating'	]]

In [12]:
#Xuất file Json
df.to_json('TLP_processing.json', orient='records', force_ascii=False)
# Exporting the DataFrame to a JSON file with pretty formatting for readability
json_output_path = '/mnt/data/TLP_processing_pretty.json'




In [13]:
# Xuất ra file CSV mới
df.to_csv('Telephone_processing.csv', index=False, encoding='utf-8-sig')